In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from src.trust_region_sqp import TrustRegionSQPFilter
import numpy as np
import casadi as ca


In [2]:
## Constants

constants = dict()
constants["gamma_0"] = 0.1
constants["gamma_1"] = 0.5
constants["gamma_2"] = 2.0
constants["eta_1"] = 1E-4 
constants["eta_2"] = 0.9
constants["mu"] = 0.01
constants["gamma_vartheta"] = 1E-4 
constants["kappa_vartheta"] = 1E-4
constants["kappa_radius"] = 0.7
constants["kappa_mu"] = 100
constants["kappa_tmd"] = 0.01

constants["init_radius"] = 1.
constants["L_threshold"] = 1.2

dataset = np.array([[0.0, 0.0], [1.0, 0.0], [0.0, 1.0], [-1.0, 0.0], [1.0, 1.0]], dtype=np.float32).T + np.array([[1.8], [2.2]], dtype=np.float32)

def cf(x:np.ndarray) -> np.ndarray: # Rosenbrock function: OF
    return 1*(x[1]-x[0]**2)**2+((x[0]-1)**2)/100

def eq_constraint(x:np.ndarray) -> np.ndarray: # equality constraints
    return x[1] - x[0]

def ineq_constraint(x:np.ndarray) -> np.ndarray: # inequality constraints
    return x[1] + x[0]

In [3]:
tr = TrustRegionSQPFilter(constants = constants, 
                          dataset = dataset, 
                          cf = cf, 
                          eqcs = [eq_constraint], 
                          ineqcs = [ineq_constraint])

tr.run()

=================0=================
poisedness = 2.1504879982195817
We have to fix the model


NameError: name 'asdfasdfsa' is not defined

In [ ]:
%debug

In [ ]:
tr.iterates[-1]

In [ ]:
tr.iterates[-2]

In [ ]:
import matplotlib.pyplot as plt
X, Y = np.meshgrid(np.linspace(-3, 3, 100),
                    np.linspace(-3, 3, 100))
levels = [0, 0.0001, 0.001, 0.1, 1]

for i in range(len(tr.iterates)):
    
    center = tr.iterates[i]['y_curr']
    radius = tr.iterates[i]['radius']
    intx = X - center[0]
    inty = Y - center[1]
    dist = np.sqrt(intx**2 + inty**2)
    intindices = dist <= radius
        
    func = cf([X, Y])
    
    func_map = tr.iterates[i]['models'].m_cf.model.model_polynomial.feval.map(X.shape[0]*X.shape[1])
    x_dm = ca.DM(X).reshape((1, X.shape[0]*X.shape[1]))
    y_dm = ca.DM(Y).reshape((1, X.shape[0]*X.shape[1]))
    # print(func_map, vertcat(x_dm, y_dm).shape)
    func_dm = func_map(ca.vertcat(x_dm, y_dm))
    func_dm = func_dm.reshape((X.shape[0], X.shape[1])).full()
    func[intindices] = func_dm[intindices]
        
    circle1 = plt.Circle(center, radius, color='black', fill=False)
    
    fig, ax = plt.subplots(1)
    ax.add_patch(circle1)
    ax.contour(X, Y, func, levels)
    x = tr.iterates[i]['Y'][0,:]
    y = tr.iterates[i]['Y'][1,:]
    plt.scatter(x, y, label=f"iteration {i}")
    plt.scatter(x[0], y[0], label=f'Best point')
    plt.plot([-3,3], [-3,3], 'b-', label=f"eq constraint")
    plt.plot([-3,3], [3,-3], 'r--', label=f"ineq constraint")
    plt.legend()
    

In [ ]:



x = [i[0] for i in tr.filter_SQP.filters]
y = [i[1] for i in tr.filter_SQP.filters]
plt.figure()
plt.scatter(x[1:], y[1:])
plt.ylabel("vartheta_k")
plt.xlabel("f_k")
